In [25]:
import nltk
tokens = nltk.word_tokenize("i am a very good boy. My name is ANkit and I live in Apartment 7, Dublin.")
print(nltk.pos_tag(tokens))

[('i', 'NN'), ('am', 'VBP'), ('a', 'DT'), ('very', 'RB'), ('good', 'JJ'), ('boy', 'NN'), ('.', '.'), ('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('ANkit', 'NNP'), ('and', 'CC'), ('I', 'PRP'), ('live', 'VBP'), ('in', 'IN'), ('Apartment', 'NNP'), ('7', 'CD'), (',', ','), ('Dublin', 'NNP'), ('.', '.')]


In [8]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [20]:
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk import pos_tag
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

In [10]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Cell_Phones_and_Accessories_5.json.gz')

df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN
1,5.0,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN
2,3.0,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN
3,2.0,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN
4,4.0,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN


In [16]:
df = df.dropna(subset = ['reviewText','summary'])
df.isna().sum()

overall                 0
verified                0
reviewTime              0
reviewerID              0
asin                    0
style              522684
reviewerName          133
reviewText              0
summary                 0
unixReviewTime          0
vote              1035232
image             1100212
dtype: int64

In [17]:
df['overall'].value_counts()

5.0    706102
4.0    184293
3.0     98189
1.0     81469
2.0     57153
Name: overall, dtype: int64

In [18]:
def cleanme(txt):
    sent = txt.lower()
    wrds = word_tokenize(sent)
    clwrds = [w for w in wrds if not w in stopwords.words('english')]
    ln = len(clwrds)
    pos = pd.DataFrame(pos_tag(wrds))
    pos = " ".join(list(pos[pos[1].str.contains("JJ")].iloc[:,0]))
    rt = [ln, " ".join(clwrds), pos]
    return(rt)

In [24]:
tmp = list()
for i in range(100):
    tmp.append(cleanme(df.iloc[i,:]['reviewText']))
tmp = pd.DataFrame(tmp)
tmp.columns = ['reviewlen', 'cleanrev', 'adjreview']

print(tmp.tail())

    reviewlen                                           cleanrev  \
95         23  works well . use travel lives luggage year . l...   
96         25  charger lasted week ad stopped charging samsun...   
97         99  charger awesome ! first n't expecting 12/12 12...   
98         17  charger woks wonderfully samsung galaxy s4 . c...   
99         13  say ? pretty much like travel chargers galaxy ...   

                                            adjreview  
95                    most low extra usb great galaxy  
96                                    .i thru several  
97  awesome first second skeptical i loose long fa...  
98                                  great recommended  
99                                          much best  


In [27]:
pd.set_option('display.max_colwidth', -1)
print(tmp.tail())


    reviewlen  \
95  23          
96  25          
97  99          
98  17          
99  13          

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    cleanrev  \
95  works well . use travel lives luggage year . low price extra usb charger works great . charges galaxy s3 note 2 .                                                                                                                                                                                                                                                                            